# Model B - CoreNLP + TFIDF

## Prepare train data

In [1]:
#java -mx3g -cp "C:\Users\zigsi\Desktop\CoreNLP\stanford-corenlp-full-2018-10-05/*" edu.stanford.nlp.naturalli.OpenIE text.txt

In [2]:
f = open("data/Weightless_dataset_train_A.csv", "r", encoding="utf-8")
first = True
questions = []
answers = []
grades = []
texts = []
for line in f:
    if first:
        first = False
        continue
    s = line.split(";")
    questions.append(s[10])
    answers.append(s[11])
    grades.append(s[14])
    texts.append(s[15])
    
print(texts[0])

Just the thought of leaving Earth both thrilled and terrified her. Her heart stopped as the trailer-sized shuttle moved forward on the track without making a sound. She took in a deep breath. she said, trying to be brave



## Prepare test data

In [13]:
f = open("data/dataset - fixed.csv", "r", encoding="utf-8")

first = True
questions_all = []
answers_all = []
grades_all = []
texts_all = []
for line in f:
    if first:
        first = False
        continue
    s = line.split(";")
    questions_all.append(s[10])
    answers_all.append(s[11])
    grades_all.append(s[14])
    texts_all.append(s[15])
    if s[15] == '1':
        print(line)
#test_answers = [answers_all[i] for i in range(len(questions_all)) if questions_all[i] == questions[0]]
#test_grades = [float(grades_all[i].replace(",", ".")) for i in range(len(questions_all)) if questions_all[i] == questions[0]]

## DATA[question_number] -> question data
old = questions_all[0]
DATA = []
data_tmp = []
for i in range(len(questions_all)):
    if questions_all[i] == old:
        triple = (questions_all[i], grades_all[i], texts_all[i])
        data_tmp.append(triple)
    else:
        old = questions_all[i]
        DATA.append(data_tmp)
        data_tmp = []
        data_tmp.append((questions_all[i], grades_all[i], texts_all[i]))
DATA.append(data_tmp)
        
print(len(DATA))
for i in DATA:
    print(len(i))

Just the thought of leaving Earth both thrilled and terrified her… Her heart stopped as the trailer-sized shuttle moved forward on the track without making a sound. She took in a deep breath. …she said, trying to be brave ….

Using its accumulated momentum, the space shuttle bulleted off the track and into Earth's orbit, leaving her home on Earth far behind.After the g-force died down, Shiranna opened her eyes and looked outside her window. She thought she was going to be able to see the entire planet from the space shuttle, but she was still too close to Earth. Though seatbelts strapped her down to her chair, her body wanted to float.

1
"She placed her left hand on her son's shoulder. ""My name is Olivia Mosby, and this is my son--Adam. Since you both look around the same age, I thought you would like to meet each other."" Shiranna looked at Adam. His cheeks were flushed."

"""Ladies and gentlemen, this is your captain speaking. There is something I would like to discuss with all of 

## TF-IDF

## Preparation

In [4]:
import requests
url = 'http://localhost:9000/?properties={"annotators": "openie,coref", "outputFormat": "json", "openie.resolve_coref": "true"}'
data = "Sarah jumped over the table. She got hurt."
response = requests.post(url, data=data)

In [5]:
for s in response.json()["sentences"]:
    for i in s["openie"]:
        print("Subject:", i["subject"], " | Relation:", i["relation"], " | Object:", i["object"])

Subject: Sarah  | Relation: jumped over  | Object: table
Subject: Sarah  | Relation: got  | Object: hurt


In [6]:
def openie_extract(text, resolve_coref=True):
    import requests
    if resolve_coref:
        url = 'http://localhost:9000/?properties={"annotators": "tokenize,ssplit,pos,lemma,openie,coref", "outputFormat": "json", "openie.resolve_coref": "true", "openie.triple.strict": "true"}'
    else:
        url = 'http://localhost:9000/?properties={"annotators": "tokenize,ssplit,pos,lemma,openie,coref", "outputFormat": "json", "openie.resolve_coref": "false", "openie.triple.strict": "true"}'
    data = text
    response = requests.post(url, data=data)
    response.encoding = "utf-8"
    triples = []
    for s in response.json()["sentences"]:
        for i in s["openie"]:
            #print("Subject:", i["subject"], " | Relation:", i["relation"], " | Object:", i["object"])
            #triples.append((lemmatize([i["subject"]])[0], lemmatize([i["relation"]])[0], lemmatize([i["object"]])[0]))
            triples.append((i["subject"], i["relation"], i["object"]))
    return triples

In [7]:
data = answers[0] + " " + texts[0]
base_triples = openie_extract(data) #+ openie_extract(data, False)
base_triples

[('Shiranna', 'feels', 'excited'),
 ('trailer-sized shuttle', 'affects', 'Shiranna heart-rate'),
 ('trailer-sized shuttle', 'even affects', 'Shiranna heart-rate'),
 ('shuttle', 'moved on', 'track'),
 ('Shiranna heart', 'stopped', 'trailer-sized shuttle moved forward'),
 ('shuttle', 'making', 'sound'),
 ('Shiranna heart', 'stopped', 'shuttle moved forward on track'),
 ('Shiranna heart', 'stopped', 'shuttle moved'),
 ('Shiranna heart', 'stopped', 'trailer-sized shuttle moved on track'),
 ('trailer-sized shuttle', 'moved forward on', 'track'),
 ('trailer-sized shuttle', 'making', 'sound'),
 ('Shiranna heart', 'stopped', 'trailer-sized shuttle moved'),
 ('trailer-sized shuttle', 'moved on', 'track'),
 ('Shiranna heart', 'stopped', 'shuttle moved on track'),
 ('shuttle', 'moved forward on', 'track'),
 ('Shiranna heart', 'stopped', 'trailer-sized shuttle moved forward on track'),
 ('Shiranna heart', 'stopped', 'shuttle moved forward'),
 ('Shiranna', 'took in', 'breath'),
 ('Shiranna', 'took 

In [8]:
import string
import nltk
from nltk.stem import WordNetLemmatizer
def getTokens(text):
    lowered = text.lower()
    table = text.maketrans({key: None for key in string.punctuation})
    lowered = lowered.translate(table)
    return nltk.word_tokenize(lowered)

def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

def preprocess(text):
    tokens = getTokens(text)
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return " ".join(lemmas)
    #pos_tag = nltk.pos_tag(lemmas)
    #print(pos_tag)
    #return " ".join([pt[0] for pt in pos_tag if pt[1] == "NN" or pt[1][0:2] == "VB" or pt[1] == "JJ"])


pre_answer = preprocess(answers[0])
pre_text = preprocess(texts[0])
#print(lemmas)

## Predicting

In [10]:
#weights = vect.transform(test_answers)

from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer() # parameters for tokenization, stopwords can be passed
tfidf = vect.fit_transform([pre_answer, pre_text])
#print("Cosine similarity between the documents: \n{}".format(cosine))
weights = vect.transform([preprocess(ta) for ta in test_answers_tfidf[0]])
predict = tfidf * weights.T

#Shiranna feels excited and scared as the shuttle is taking off and it even affects her heart-rate and her temperature.
#test_answers = ["Excited, scared. It affected her heart rate and temp"]

correct = 0
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(test_answers)):
    test_answer = test_answers[i]
    triples = openie_extract(test_answer.encode("utf8"))
    
    p = 0
    p_tfidf = 0
    p_triples = 0
    
    #if len(triples) < 1:
    prediction = max(predict[0,i], predict[1,i])
    if prediction > 0.35: #0.5
        p_tfidf = 1
    elif prediction > 0.2: #0.3
        p_tfidf = 0.5
    else:
        p_tfidf = 0
    #else:
    for bt in base_triples:
        for t in triples:
            if t[0] == bt[0] or t[1] == bt[1] or t[2] == bt[2]:
                p_triples += 0.5
    p_triples = p_triples*4 / max(len(triples), 1)
    prediction = 0
    if p_triples >= 1:
        p_triples = 1
    elif p_triples >= 0.5:
        p_triples = 0.5
    else:
        p_triples = 0
        
    p = (p_triples + p_tfidf) / 2
    if p > 0 and p < 1 and p != 0.5:
        p = 0.5
    
    print("Predicted:", p, " Real:", test_grades[i], " --- ", len(triples))
    
    if p == test_grades[i]:
        correct += 1
        if p == 1 or p == 0.5:
            TP += 1
        else:
            TN += 1
    else:
        print(len(triples))
        if p == 1 or p == 0.5:
            FP += 1
        else:
            FN += 1
    
print("Correct: ", correct, "/", len(test_answers))

Predicted: 0.5  Real: 0.5  ---  5
Predicted: 1.0  Real: 1.0  ---  2
Predicted: 0.5  Real: 0.0  ---  1
1
Predicted: 0.5  Real: 0.5  ---  0
Predicted: 0.5  Real: 1.0  ---  0
0
Predicted: 0.5  Real: 0.5  ---  0
Predicted: 1.0  Real: 0.5  ---  1
1
Predicted: 0.5  Real: 0.5  ---  2
Predicted: 0.5  Real: 0.5  ---  0
Predicted: 0.5  Real: 1.0  ---  0
0
Predicted: 0.5  Real: 1.0  ---  1
1
Predicted: 1.0  Real: 0.5  ---  13
13
Predicted: 0.5  Real: 0.5  ---  0
Predicted: 0.5  Real: 0.5  ---  1
Predicted: 0.5  Real: 0.5  ---  2
Predicted: 1.0  Real: 1.0  ---  3
Predicted: 1.0  Real: 1.0  ---  3
Predicted: 1.0  Real: 1.0  ---  8
Predicted: 0.5  Real: 1.0  ---  0
0
Predicted: 1.0  Real: 1.0  ---  11
Predicted: 0.5  Real: 0.5  ---  2
Predicted: 0.5  Real: 0.5  ---  6
Predicted: 1.0  Real: 1.0  ---  1
Predicted: 1.0  Real: 1.0  ---  1
Predicted: 0.0  Real: 0.5  ---  0
0
Predicted: 0.5  Real: 0.5  ---  2
Predicted: 1.0  Real: 0.5  ---  8
8
Predicted: 0.5  Real: 0.0  ---  0
0
Predicted: 0.5  Real: 0.5

[texts[0] + answers[0]] 0.3 0.2 -> 38/81 correct

[texts[0], answers[0]]  0.3 0.2 -> 42/81 correct 

CoreNLP openie+coref /w TFIDF -> 32/81 correct

CoreNLP openie /w TFIDF ->  40/81 correct

CoreNLP openie relative scoring /w TFIDF -> 36/81 correct

CoreNLP openie+coref relative scoring /w TFIDF -> 33/81 correct

CoreNLP (not strict) openie+coref relative scoring AND TFIDF | AVG -> 43/81 correct

CoreNLP openie+coref relative scoring AND TFIDF | AVG -> 40/81 correct

CoreNLP openie+coref(with and without) relative scoring AND TFIDF | AVG -> 41/81 correct

## F1 Scoring

In [ ]:
T = TP + TN
A = T + FP + FN
P = TP/(TP+FP)
R = TP/(TP+FN)
F1 = 2*P*R/(P+R)
print("Classification Accuracy: ", T, "/", A, " = ", T/A)
print("Precision: ", TP, "/", TP+FP, " = ", P)
print("Recall: ", TP, "/", TP+FN, " = ", TP/(TP+FN))
print("F1: ", 2*P*R, "/", P+R, " = ", F1)